In [5]:
from poisson_fem import PoissonFEM
import ROM
import numpy as np
import torch
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [6]:
lin_dim_rom = 16
mesh = PoissonFEM.RectangularMesh(torch.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

In [7]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
def essBoundaryFun(x):
    return 1.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

In [8]:
def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

In [9]:
#Define boundary flux field
def flux(x):
    a = np.array([1, 2, 3])
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [10]:
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)

In [11]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [12]:
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace, ksp)

In [13]:
rhs.setRhsStencil(mesh, K)

In [14]:
# Preallocate
x = PETSc.Vec().createSeq(mesh.nEq)

In [15]:
# define rom
rom = ROM.ROM(mesh, K, rhs)

In [16]:
N = 1e5

In [17]:
# lmbda = PETSc.Vec().createSeq(mesh.nCells)
x = torch.ones(lin_dim_rom**2)
start = time.time()
for n in range(int(N)):
#     lmbda.setValues(range(mesh.nCells), np.ones(lin_dim_rom**2))
#     lmbda.array = x
    lmbda = PETSc.Vec()
    lmbda.createWithArray(x)
    
    # solve implies stiffness/ rhs assembly and solution
    rom.solve(lmbda)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  0.0002159498167037964


In [ ]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [ ]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ lmbda.array
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    rhs.assemble(lmbda)   # for fair comparison
    x1 = lg.spsolve(K1, rhs.vector.array)
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

In [ ]:
diff = np.linalg.norm(rom.solution.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

In [ ]:
# plot
rom.plotSolution()

In [ ]:
start = time.time()
for n in range(int(N)):    
    # solveAdjoint implies adjoint solution only. No assembly implied!!
    rom.solveAdjoint(rom.rhs.vector)
adjoint_time = (time.time() - start)/N
print('Adjoint time = ', adjoint_time)

In [ ]:
diff = np.linalg.norm(rom.adjointSolution.array - rom.solution.array)/\
        np.linalg.norm(rom.solution.array)   # needs to be 0 if K is symmetric
print('Adjoint difference == ', diff)

In [ ]:
rhsAdj = PETSc.Vec().createSeq(rom.mesh.nEq)
rhsAdj.setValues(range(rom.mesh.nEq), np.arange(1.0, rom.mesh.nEq + 1))
rom.solveAdjoint(rhsAdj)

In [ ]:
#assembly only
start = time.time()
for n in range(int(N)):    
    rom.stiffnessMatrix.assemble(lmbda)
assemble_time = (time.time() - start)/N
print('Assembly time = ', assemble_time)

In [ ]:
#setOperators only
start = time.time()
for n in range(int(N)):    
    rom.stiffnessMatrix.solver.setOperators(rom.stiffnessMatrix.matrix)
setOperator_time = (time.time() - start)/N
print('setOperator time = ', setOperator_time)

In [ ]:
rom.solution.array

In [ ]:
lmbda.createWithArray(np.exp(lmbda))

In [ ]:
lmbda.array

In [18]:
lmbda = PETSc.Vec()
lmbda.createWithArray(x)

In [47]:
K.globStiffGrad[0]

<288x288 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [42]:
import scipy.sparse as sps
K0_dense = sps.csr_matrix.todense(K.globStiffGrad[0])
K0_dense

matrix([[0.66666667, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [113]:
import scipy.sparse as sps
K0 = sps.csr_matrix.todense(K.globStiffGrad[0])
K1 = sps.csr_matrix.todense(K.globStiffGrad[1])
K0t = torch.tensor(K0)

In [137]:
Kt = torch.empty((288, 256, 288))

In [138]:
Kt.shape

torch.Size([288, 256, 288])

In [139]:
Kt[:, 0, :] = K0t

In [142]:
a, b = [1, 2]

In [146]:
torch.tensor(lmbda.array)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [149]:
x = torch.randn((3, 4))

In [150]:
x

tensor([[ 0.7841,  0.3695, -0.1776,  0.1481],
        [-0.6056, -0.3860, -0.0645, -1.5107],
        [ 0.0398, -0.3348,  0.0934,  0.5923]])

In [151]:
x.t()

tensor([[ 0.7841, -0.6056,  0.0398],
        [ 0.3695, -0.3860, -0.3348],
        [-0.1776, -0.0645,  0.0934],
        [ 0.1481, -1.5107,  0.5923]])